# BBC Text MultiClass Classification

In [166]:
# load packages
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn import decomposition, ensemble
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas, xgboost, numpy, textblob, string
import pandas as pd
import numpy as np

In [167]:
# load the dataset
trainDF = pd.read_csv('E:/emil/code/data/olddataset.csv') # encoding = "latin"

In [168]:
#mengacak data
ndata = len(trainDF)
def sampel_data(data,n):
    return data.sample(frac=1).reset_index(drop=True) # mengacak seluruh data
trainDF = sampel_data(trainDF,ndata)
print(len(trainDF))

5018


## Data preparation

In [169]:
def get_label(train_y,valid_y):
    train_labels = train_y
    valid_labels = valid_y
    # label encode the target variable 
    encoder = preprocessing.LabelEncoder()
    train_y = encoder.fit_transform(train_y)
    valid_y = encoder.fit_transform(valid_y)
    return train_y,valid_y

## Feature Extraction

### TF-IDF Vectors

In [170]:
def tfidf(text,train_x,valid_x):
    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=50000)
    tfidf_vect.fit(text)
    xtrain_tfidf =  tfidf_vect.transform(train_x)
    xvalid_tfidf =  tfidf_vect.transform(valid_x)
    return xtrain_tfidf,xvalid_tfidf

### Model Building

In [171]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    return metrics.accuracy_score(predictions, valid_y)

#### Naive Bayes

In [172]:
from keras.losses import categorical_crossentropy
from sklearn.model_selection import KFold

In [173]:
text = trainDF['text']
label = trainDF['category']

In [174]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
akurasi = []

for train, test in kfold.split(text, label):
    train_y,valid_y = get_label(label[train],label[test])
    xtrain_tfidf,xvalid_tfidf = tfidf(trainDF['text'],text[train],text[test])
    accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
    akurasi.append(accuracy)
    
    # Increase fold number
    fold_no = fold_no + 1

In [175]:
# == meanmpilkan akurasi tiap fold dan rata-rata seluruhnya ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(akurasi)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Akurasi: {akurasi[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(akurasi)} (+- {np.std(akurasi)})')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Akurasi: 0.7589641434262948%
------------------------------------------------------------------------
> Fold 2 - Akurasi: 0.7091633466135459%
------------------------------------------------------------------------
> Fold 3 - Akurasi: 0.7529880478087649%
------------------------------------------------------------------------
> Fold 4 - Akurasi: 0.7908366533864541%
------------------------------------------------------------------------
> Fold 5 - Akurasi: 0.8067729083665338%
------------------------------------------------------------------------
> Fold 6 - Akurasi: 0.7470119521912351%
------------------------------------------------------------------------
> Fold 7 - Akurasi: 0.8087649402390438%
------------------------------------------------------------------------
> Fold 8 - Akurasi: 0.7410358565737052%
--------